In [54]:
import requests

In [55]:
from bs4 import BeautifulSoup
import pandas as pd
import time

In [56]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [57]:
data = requests.get(standings_url)

In [63]:
soup = BeautifulSoup(data.text)

In [66]:
standings_table = soup.select('table.stats_table')[0]

IndexError: list index out of range

In [ ]:
links = standings_table.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if "/squads/" in l]

In [ ]:
teams_url = [f"https://fbref.com{l}" for l in links]

In [ ]:
Mancity = teams_url[0]

In [ ]:
data = requests.get(team_url)

In [ ]:
player_stats = pd.read_html(data.text, match="Standard Stats")

In [ ]:
player_stats[0]

In [ ]:
westham = team_urls[1]
data = requests.get(westham)
player_stats = pd.read_html(data.text, match="Standard Stats")[0]

In [ ]:
years = list(range(2023,2021,-1))

In [ ]:
years

In [ ]:
all_matches = []

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if "/squads/" in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com/{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats","").replace("-","")
        
        data = requests.get(team_url)
        player_stats = pd.read_html(data.text, match="Standard Stats")[0]
        
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and "all_comps/shooting/" in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        shooting = pd.read_html(data.text, match = "Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        
        try:
            team_data = player_stats.merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]])
        except ValueError:
            pass

        team_data['Season'] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)


In [ ]:
matches_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df.to_csv("matches.csv")